# **PINKSHEPZ'S 07 DATA ANALYTIC**
## Dataset
1. *Candlestick price*: XAUUSD TF 1D - 12 Oct 2020 to 8 May 2023
2. *Economic figure report*: FOREX FACTORY TF 1D - 13 Oct 2020 to 31 Dec 2022

## Analytics
1. Candlestick price distribution
    * Scatter 3D: Time - %Change - Frequency distribution
    * Line 2D: Timezone - Mean %Change
    * Scatter 2D: %Change of n candlestick - %Change of n+1 candlestick
2. Economic figure distribution
3. Candlestick price x economic figure correlation

## **01 IMPORT & DATA PREPARATION**

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

In [8]:
# Import dataset
PATH_XAUUSD = '/workspaces/Enterprise/03_Analytics/input/XAUUSD-1H_220103_230503.csv'
PATH_FOREX = '/workspaces/Enterprise/03_Analytics/input/FOREX_311222_131020.csv'

DF_XAUUSD = pd.read_csv(PATH_XAUUSD).iloc[:, 1:]
DF_XAUUSD.iloc[:, 0] = pd.to_datetime(DF_XAUUSD.iloc[:, 0]).dt.date
for col in DF_XAUUSD.columns[1:]:
    DF_XAUUSD[col] = DF_XAUUSD[col].astype('float64')

DF_FOREX = pd.read_csv(PATH_FOREX).iloc[:, 1:]
DF_FOREX.iloc[:, 0] = pd.to_datetime(DF_XAUUSD.iloc[:, 0]).dt.date
DF_FOREX['currency'].astype('category')
DF_FOREX['impact'].astype('category')
DF_FOREX['event'].astype('category')

def handle_number_format(number: str) -> list:
    """Return [number as float, unit i.e. % K M]"""
    # Case 1: na
    if number == '':
        return [None, None]
    number = str(number)
    # Case 2: % value
    if (number[-1] == '%') & (number.lstrip('-')[0].isnumeric() == True):
        return [float(number.rstrip('%')), '%']
    # Case 3: 1K 1M 1B 1T unti
    if (number[-1].isalpha() == True) & (number.lstrip('-')[0].isnumeric() == True):
        return [float(str(number)[:-1]), number[-1]]
    # Case 4: ordinary numbers
    if number.replace(',', '').replace('.', '').lstrip('-').isnumeric() == True: 
        return [float(number.replace(',', '')), None]
    # Case 5: non-numbers
    return [number, None]

for col in DF_FOREX.columns[5:]:
    DF_FOREX[col] = DF_FOREX[col].fillna('').apply(handle_number_format)

## **02 CANDLESTICK EDA**

In [11]:
DF_XAUUSD.describe()

,open,high,low,close,%Change
count,8271.000000,8271.000000,8271.000000,8271.000000,8271.000000
mean,1830.425790,1832.880063,1827.481012,1830.438785,0.000833
std,99.467486,99.720986,99.318595,99.490533,0.193789
min,1616.560000,1620.440000,1614.090000,1616.670000,-1.918800
25%,1760.360000,1762.685000,1757.560000,1760.360000,-0.090750
50%,1828.710000,1831.330000,1825.880000,1828.670000,-0.004100
75%,1912.510000,1915.045000,1908.420000,1912.470000,0.089900
max,2069.480000,2070.430000,2051.050000,2069.320000,2.020200
